In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

warnings.filterwarnings('ignore')

DATA WRANGLING

Importing the Data

In [2]:
data = pd.read_csv('learningSet.csv.zip')

Exploring the Data

We will check the data types, unique values, null values and statistical overview of the data

Data Overview

In [3]:
data.shape


(95412, 481)

In [4]:
data.describe()

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
count,95412.000000,95412.000000,95412.000000,71747.000000,12386.000000,74126.000000,50680.000000,95412.000000,42558.000000,42558.000000,...,95412.000000,85439.000000,85439.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95280.000000
mean,9141.363256,54.223117,2723.602933,61.611649,1.527773,3.886248,5.345699,3.321438,0.152075,0.059166,...,9135.651648,9151.022917,8.093739,13.347786,95778.176959,0.050759,0.793073,0.500618,1.910053,31.533711
std,343.454752,953.844476,2132.241295,16.664199,0.806861,1.854960,2.742490,9.306899,0.470023,0.262078,...,320.394019,294.257260,8.213242,10.769997,55284.596094,0.219506,4.429725,0.500002,1.072749,18.764614
min,8306.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7211.000000,0.000000,1.285714,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,8801.000000,0.000000,201.000000,48.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,...,8810.000000,8903.000000,4.000000,8.384615,47910.750000,0.000000,0.000000,0.000000,1.000000,15.000000
50%,9201.000000,1.000000,2610.000000,62.000000,1.000000,4.000000,6.000000,0.000000,0.000000,0.000000,...,9201.000000,9204.000000,6.000000,11.636364,95681.500000,0.000000,0.000000,1.000000,2.000000,32.000000
75%,9501.000000,2.000000,4601.000000,75.000000,2.000000,5.000000,8.000000,3.000000,0.000000,0.000000,...,9409.000000,9409.000000,11.000000,15.477955,143643.500000,0.000000,0.000000,1.000000,3.000000,49.000000
max,9701.000000,72002.000000,9710.000000,98.000000,7.000000,7.000000,9.000000,241.000000,6.000000,4.000000,...,9603.000000,9702.000000,1088.000000,1000.000000,191779.000000,1.000000,200.000000,1.000000,4.000000,62.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 481 entries, ODATEDW to GEOCODE2
dtypes: float64(97), int64(310), object(74)
memory usage: 350.1+ MB


Checking the Null Values

In [6]:
data.isna().sum()

ODATEDW       0
OSOURCE       0
TCODE         0
STATE         0
ZIP           0
           ... 
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
CLUSTER2    132
GEOCODE2    132
Length: 481, dtype: int64

In [7]:
def check_nan(df):
    nulls = pd.DataFrame((df.isna().sum()/len(df))*100)
    nulls = nulls.reset_index()
    nulls.columns = ['column_name', 'percentage_null_values']
    nulls.sort_values(by='percentage_null_values', ascending = False, inplace = True)
    return nulls

In [8]:
nulls = check_nan(data)
nulls

,column_name,percentage_null_values
414,RDATE_5,99.990567
436,RAMNT_5,99.990567
412,RDATE_3,99.746363
434,RAMNT_3,99.746363
413,RDATE_4,99.705488
...,...,...
168,ETHC3,0.000000
167,ETHC2,0.000000
166,ETHC1,0.000000
165,HHD12,0.000000


In [9]:
only_nulls = nulls[nulls['percentage_null_values']>0]
only_nulls.describe()

,percentage_null_values
count,92.000000
mean,58.770249
std,32.281668
min,0.138347
25%,32.986941
50%,63.800675
75%,87.160944
max,99.990567


We create a list where we will put the columns we want to drop

OSOURCE - symbol definitions not provided, too many categories \ ZIP CODE - we are including state already



In [11]:
data['ZIP']


0        61081
1        91326
2        27017
3        95953
4        33176
         ...  
95407    99504
95408    77379
95409    48910
95410    91320
95411    28409
Name: ZIP, Length: 95412, dtype: object

In [12]:
drop_1 = data[['OSOURCE','ZIP']]
drop_list = drop_1.columns.tolist()
drop_list

['OSOURCE', 'ZIP']

Identify columns that over 85% missing values

In [13]:
eighty_five = nulls[nulls['percentage_null_values']>=85]
eighty_five

,column_name,percentage_null_values
414,RDATE_5,99.990567
436,RAMNT_5,99.990567
412,RDATE_3,99.746363
434,RAMNT_3,99.746363
413,RDATE_4,99.705488
435,RAMNT_4,99.705488
437,RAMNT_6,99.186685
415,RDATE_6,99.186685
446,RAMNT_15,92.388798
424,RDATE_15,92.388798


In [14]:
for name in eighty_five.column_name:
    drop_list.append(name)
    
drop_list

['OSOURCE',
 'ZIP',
 'RDATE_5',
 'RAMNT_5',
 'RDATE_3',
 'RAMNT_3',
 'RDATE_4',
 'RAMNT_4',
 'RAMNT_6',
 'RDATE_6',
 'RAMNT_15',
 'RDATE_15',
 'RDATE_23',
 'RAMNT_23',
 'RDATE_20',
 'RAMNT_20',
 'RAMNT_7',
 'RDATE_7',
 'RAMNT_17',
 'RDATE_17',
 'RDATE_21',
 'RAMNT_21',
 'RAMNT_10',
 'RDATE_10',
 'RDATE_13',
 'RAMNT_13',
 'NUMCHLD']

Removing those columns from the dataframe

In [16]:
df = data.drop(columns=drop_list, axis = 1)
nulls_2 = check_nan(df)
nulls_2 = nulls_2[nulls_2['percentage_null_values']>0]
nulls_2

,column_name,percentage_null_values
421,RAMNT_11,84.551209
411,RDATE_11,84.551209
426,RAMNT_19,83.359535
416,RDATE_19,83.359535
410,RDATE_9,82.461326
...,...,...
192,MSA,0.138347
193,ADI,0.138347
194,DMA,0.138347
453,GEOCODE2,0.138347


In [17]:
nulls_2.describe()

,percentage_null_values
count,67.000000
mean,45.921130
std,28.493307
min,0.138347
25%,21.814342
50%,55.395548
75%,73.064185
max,84.551209


Data Cleaning

Cleaning Gender

In [18]:
df.GENDER.value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [19]:
df['GENDER'] = np.where(df['GENDER'].isin(['F','M']), df['GENDER'], "other")
df.GENDER.value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64

Checking and cleaning the Nans in GENDER

No nulls in the column

In [20]:
df.GENDER.isna().sum()

0

# Lab | Feature engineering

In [21]:
numericals = df.select_dtypes(np.number)
numericals.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.0,NaN,NaN,0,NaN,NaN,NaN,...,8911,9003.0,4.0,7.741935,95515,0,0.0,0,4,39.0
1,9401,1,5202,46.0,6.0,9.0,16,0.0,0.0,3.0,...,9310,9504.0,18.0,15.666667,148535,0,0.0,0,2,1.0
2,9001,1,0,NaN,3.0,1.0,2,0.0,0.0,1.0,...,9001,9101.0,12.0,7.481481,15078,0,0.0,1,4,60.0
3,8701,0,2801,70.0,1.0,4.0,2,0.0,0.0,0.0,...,8702,8711.0,9.0,6.812500,172556,0,0.0,1,4,41.0
4,8601,0,2001,78.0,3.0,2.0,60,1.0,0.0,9.0,...,7903,8005.0,14.0,6.864865,7112,0,0.0,1,2,26.0


In [22]:
n_nulls = check_nan(numericals)
n_nulls.describe()

,percentage_null_values
count,382.000000
mean,8.053867
std,21.128976
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,84.551209


In [23]:
[len(n_nulls[n_nulls['percentage_null_values']>0]), len(n_nulls[n_nulls['percentage_null_values']==0])]

[66, 316]

In [24]:
only_n_nulls = n_nulls[n_nulls['percentage_null_values']>0]
null_columns = only_n_nulls.column_name.to_list()
only_n_nulls

,column_name,percentage_null_values
345,RDATE_11,84.551209
355,RAMNT_11,84.551209
360,RAMNT_19,83.359535
350,RDATE_19,83.359535
354,RAMNT_9,82.461326
...,...,...
316,ADATE_3,2.043768
151,DMA,0.138347
150,ADI,0.138347
149,MSA,0.138347


In [25]:
only_n_nulls.describe()

,percentage_null_values
count,66.000000
mean,46.614809
std,28.135797
min,0.138347
25%,22.291483
50%,55.395548
75%,73.064185
max,84.551209


In [26]:
numericals.corrwith(numericals['TARGET_D']).sort_values(ascending=False)

TARGET_D    1.000000
TARGET_B    0.774232
RAMNT_9     0.090168
RAMNT_14    0.084498
RAMNT_22    0.081324
              ...   
IC15       -0.025528
EC4        -0.026064
CLUSTER2   -0.029087
ADATE_5          NaN
ADATE_15         NaN
Length: 382, dtype: float64

# Cleaning GEOCODE2

In [27]:
df.GEOCODE2.value_counts()

A    34484
B    28505
D    16580
C    15524
       187
Name: GEOCODE2, dtype: int64

In [28]:
df.GEOCODE2.unique()

array(['C', 'A', 'D', 'B', ' ', nan], dtype=object)

In [29]:
df.GEOCODE2.isna().sum()

132

In [30]:
round(len(df[df['GEOCODE2'] == ' '])/df.shape[0]*100,2), round(df.GEOCODE2.isna().sum()/df.shape[0]*100,2)

(0.2, 0.14)

In [31]:
def drop_fill(df, cols = []):
    for col in cols:
        _ = round(len(df[df[col] == ' '])/df.shape[0]*100,2)
        print(_, ' blank values')
        if _ < 5:
            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True)
            n = round(df[col].isna().sum()/df.shape[0]*100,2)
            print(n, 'NaN values')
            if n >= 5:
                df[col].fillna(mode, inplace=True)
            else:
                df = df[~df[col].isna()]
    return df

In [32]:
df = drop_fill(df, cols = ['GEOCODE2'])
df.GEOCODE2.value_counts()

0.2  blank values
0.33 NaN values


A    34484
B    28505
D    16580
C    15524
Name: GEOCODE2, dtype: int64

In [33]:
df.GEOCODE2.unique()

array(['C', 'A', 'D', 'B'], dtype=object)

# Cleaning WEALTH1

In [34]:
df.WEALTH1.unique()

array([nan,  9.,  1.,  4.,  2.,  6.,  0.,  5.,  8.,  3.,  7.])

In [35]:
df.WEALTH1.value_counts()

9.0    7578
8.0    6784
7.0    6192
6.0    5816
5.0    5272
4.0    4807
3.0    4230
2.0    4080
1.0    3450
0.0    2409
Name: WEALTH1, dtype: int64

In [36]:
round(df.WEALTH1.isna().sum()/df.shape[0]*100,2)


46.77

# Replacing NaN with KNN Imputation
Define imputer

In [37]:
df2 = df[['WEALTH1','HIT']].copy()
df2.head()

,WEALTH1,HIT
0,NaN,0
1,9.0,16
2,1.0,2
3,4.0,2
4,2.0,60


In [38]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputed = imputer.fit_transform(df2)
df_imputed = pd.DataFrame(imputed, columns=df2.columns)
df_imputed['WEALTH1'].unique()

array([6.2, 9. , 1. , 4. , 2. , 6. , 0. , 5. , 8. , 3. , 7. ])

In [39]:
df_imputed['WEALTH1'].value_counts()

6.2    44475
9.0     7578
8.0     6784
7.0     6192
6.0     5816
5.0     5272
4.0     4807
3.0     4230
2.0     4080
1.0     3450
0.0     2409
Name: WEALTH1, dtype: int64

In [40]:
df['WEALTH1'] = df_imputed['WEALTH1']
df['WEALTH1'].value_counts()

6.2    44319
9.0     7562
8.0     6763
7.0     6178
6.0     5802
5.0     5252
4.0     4788
3.0     4212
2.0     4065
1.0     3435
0.0     2399
Name: WEALTH1, dtype: int64

CLEANING ADI, MSA ans DMA

In [41]:
df.ADI.unique()

array([177.,  13., 281.,  67., 127., 185.,  91., 251., 391., 181., 269.,
        83., 107., 249., 291.,  57., 227., 329., 351.,  55., 133.,  51.,
        73., 159., 201.,  75., 233., 591., 203., 175.,  59., 105., 111.,
       173., 415., 323., 187.,  65., 412., 109., 425., 405., 197., 279.,
       645., 207.,  89., 361., 209., 213.,  15.,  21., 113., 131., 469.,
       199., 129., 441., 459., 235., 301.,  39., 429., 285., 245., 335.,
       393., 577., 439., 455., 383., 263.,   0., 427., 307., 275., 119.,
       229., 273., 325., 371., 377., 315., 342., 421., 219.,  61.,  93.,
       375., 283., 381., 243., 353., 613., 339., 367., 417., 299., 271.,
       157.,  77., 327.,  53.,  87., 451., 627.,  71., 253., 363., 403.,
       115., 389.,  63., 321., 448., 237., 217., 241., 123., 221., 165.,
       277.,  85., 319., 247.,  69., 379.,  95.,  17., 625., 462., 303.,
       179., 331., 231., 337., 257., 409., 457., 359., 373., 413., 215.,
       205., 313., 211., 305., 355., 419., 473., 42

In [42]:
df.MSA.unique()

array([   0., 4480., 9340., 5000., 2030., 3960., 5360., 3480.,  760.,
       2160., 5960., 5120.,  440., 6580., 1600.,  680., 3360., 7040.,
       2920., 3120., 6440.,  640., 6120., 3000., 7600., 2680., 5080.,
       1360., 2180., 4420., 1960., 6640., 7360., 5945., 1920., 1145.,
       7520., 2800., 1520., 7510., 1620., 4520., 7320.,  720., 4720.,
       1560., 8960.,   40., 6720.,  520., 9360., 3710., 8280., 6760.,
       3160., 5560., 3600., 6780., 5800., 4120., 5160., 5880.,  380.,
       3440., 2710., 2020., 9040., 6200., 6880., 6920., 2760., 7080.,
       4080., 2580., 4100., 5790., 4200.,  600., 4680., 4040., 1640.,
       5775., 8720., 3980., 2240., 1720., 5910., 9260.,  200., 7400.,
       1020., 3290., 2720., 7240., 3800., 8120.,  870., 2320., 3760.,
       1040., 2840., 3880.,  920., 6080.,  460.,  320., 1400., 3320.,
       2640., 8200., 4890., 3660., 2670., 1000., 2520., 2440., 8520.,
       3720., 2080., 7500., 3920., 1125., 4400.,  840., 7485., 8750.,
       8735., 1150.,

In [43]:
df.DMA.unique()

array([682., 803., 518., 862., 528., 691., 509., 643., 624., 659., 671.,
       527., 613., 716., 770., 505., 717., 534., 560., 547., 571., 602.,
       800., 638., 618., 609., 820., 821., 635., 675., 563., 819., 617.,
       637., 606., 709., 632., 807., 698., 623., 507., 627., 524., 517.,
       539., 649., 868., 546., 529., 567., 825., 512., 669., 764., 575.,
       548., 662., 811., 801., 652., 771., 603., 556., 622., 561., 724.,
       804., 633., 839., 686., 650., 743., 619., 678., 753., 610., 745.,
       604., 749., 670., 765., 711., 553., 762., 520., 503., 551., 515.,
       647., 544., 676., 752., 545., 754., 810., 790., 656., 755., 641.,
       616., 628., 588., 581., 540., 658., 583., 866., 642., 746., 634.,
       648., 705., 725., 744., 513., 612., 813., 531., 751., 500., 630.,
       611., 789., 582., 693., 692., 828., 626., 710., 542., 855., 687.,
       679., 640., 722., 620., 881., 564., 522., 756., 570., 718., 530.,
       557., 625., 541., 657., 631., 550., 525., 60

In [ ]:
df.ADI.isna().sum(), df.DMA.isna().sum(), df.MSA.isna().sum()

# Use appropriate EDA technique where ever necessary.
REMOVING COLUMNS
Variance Threshold - Only for Numericals

Removing the ones with low Variance.

We first isolate the targets the targets

In [44]:
targets = df[['TARGET_B', 'TARGET_D']]
numericals = numericals.drop(['TARGET_B', 'TARGET_D'], axis=1)
targets.value_counts()

TARGET_B  TARGET_D
0         0.00        90263
1         10.00         938
          15.00         589
          20.00         576
          5.00          503
                      ...  
          44.21           1
          16.87           1
          17.50           1
          18.25           1
          4.50            1
Length: 71, dtype: int64

In [47]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

num_v = variance_threshold_selector(numericals, 0.9)

NameError: name 'VarianceThreshold' is not defined

Looking at the removed columns:

In [48]:
removed_columns = pd.DataFrame(data=(numericals.columns,selection.variances_,selection.get_support()), index=('column_name','variance','statement')).T
removed_columns[removed_columns['statement'] == False]

NameError: name 'selection' is not defined

# Recursive feature elimination: scoring method, ranking the most important - for Categoricals and Numericals

It's not for target, only for columns.

Ideal step = 1

from sklearn.feature_selection import RFE from sklearn import linear_model

lm = linear_model.LinearRegression() rfe = RFE(lm, n_features_to_select=20, verbose=False, step=1) rfe.fit(num_v, targets)

df = pd.DataFrame(data = rfe.ranking_, columns=['Rank']) df['Column_name'] = numerical.columns df[df['Rank']==1]

In [49]:
num_v.head()

NameError: name 'num_v' is not defined

In [ ]:
num_v.corrwith(targets['TARGET_D']).sort_values(ascending=False)

# Preparing data for EDA
Separating categorical data

In [50]:
categoricals = data.select_dtypes(np.object)
categoricals.head()

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


Separating discrete and continuous data

In [51]:
def separate_num(df, count):
    continuous = []
    discrete = []
    for col in df:
        if len(df[col].value_counts()) > count:
            continuous.append(col)
        else:
            discrete.append(col)
    print(len(continuous), ' continuous numerical columns')
    print(len(discrete), 'discrete numerical columns')
    continuous_df = df[continuous]
    discrete_df = df[discrete]
    return continuous_df, discrete_df            

In [52]:
continuous_df, discrete_df = separate_num(num_v, 100)

NameError: name 'num_v' is not defined

Checking the continuous and discrete data that are the most correlated to the target

In [53]:
continuous_df.corrwith(targets['TARGET_D']).sort_values(ascending=False).head(10)
top10_c_D = pd.DataFrame(continuous_df.corrwith(targets['TARGET_D']).sort_values(ascending=False).head(10))
top10_c_D.columns = ['correlation']
top10_c_D

NameError: name 'continuous_df' is not defined

In [54]:
continuous_df.corrwith(targets['TARGET_B']).sort_values(ascending=False).head(10)
top10_c_B = pd.DataFrame(continuous_df.corrwith(targets['TARGET_B']).sort_values(ascending=False).head(10))
top10_c_B.columns = ['correlation']
top10_c_B

NameError: name 'continuous_df' is not defined

In [55]:
discrete_df.corrwith(targets['TARGET_B']).sort_values(ascending=False).head(10)
top10_d_B = pd.DataFrame(discrete_df.corrwith(targets['TARGET_B']).sort_values(ascending=False).head(10))
top10_d_B.columns = ['correlation']
top10_d_B

NameError: name 'discrete_df' is not defined

In [ ]:
discrete_df.corrwith(targets['TARGET_D']).sort_values(ascending=False).head(10)
top10_d_D = pd.DataFrame(discrete_df.corrwith(targets['TARGET_D']).sort_values(ascending=False).head(10))
top10_d_D.columns = ['correlation']
top10_d_D

In [ ]:
CB = top10_c_B.index.to_list()
CD = top10_c_D.index.to_list()
DB = top10_d_B.index.to_list()
DD = top10_d_D.index.to_list()

In [ ]:
def check_dist(df, l):
    for column in df:  
        if column in l:
            plt.figure(figsize=(10,5))
            plt.xlabel(column)
            plt.ylabel('Density')
            sns.kdeplot(df[column],shade=True)
            plt.show()
    return plt.show()

In [ ]:
dist1 = check_dist(continuous_df, CB)

In [ ]:
Top 10 correlated with TARGET_D

In [ ]:
dist2 = check_dist(continuous_df, CD)

# Boxen plots
Top 10 correlated with TARGET_B

In [ ]:
for column in continuous_df:      
    if column in CB:
        plt.figure(figsize=(6,3))
        plt.ylabel('Amounts')
        sns.boxenplot(x=continuous_df[column])
        plt.show()

In [ ]:
Top 10 correlated with TARGET_D

In [ ]:
for column in continuous_df:      
    if column in CD:
        plt.figure(figsize=(6,3))
        plt.ylabel('Amounts')
        sns.boxenplot(x=continuous_df[column])
        plt.show()

Heatmap of the correlation matric for the most correlated variables with Target B - continuous data

In [56]:
def heatmap(df, l, targets):
    matrix = pd.concat([df[df.columns.intersection(l)], targets], axis=1)
    mask = np.zeros_like(matrix.corr())
    mask[np.triu_indices_from(mask)] = True
    fig, ax = plt.subplots(figsize=(12, 8))
    ax = sns.heatmap(matrix.corr(), mask=mask, annot=True, cmap='Blues')
    plt.show()

In [57]:
m1 = heatmap(continuous_df, CB, targets)

NameError: name 'continuous_df' is not defined

There are correlations higher than 0.9 between the IC variables and also between Hv1 and HV2. This level of colinearity will badly affect the model we will have to keep the colinear variables that are the most correlated to the target variables.

Heatmap of the correlation matric for the most correlated variables with Target D - continuous data

In [ ]:
m2 = heatmap(continuous_df, CD, targets)

In [58]:
The RAMNT variables are all quite colinear with each other, especially RAMNT_12 and RAMNT_14. It might not be needed to keep all of them. Same observation for LASTGIFT and AVGGIFT, which on top of being highly correlated with each other are very correlated to the RAMNT variables as well.

SyntaxError: invalid syntax (Temp/ipykernel_51040/4040415533.py, line 1)

In [59]:
for column in discrete_df:
    if column in DD:
        plt.hist(discrete_df[column], bins=10, edgecolor="blue", color='purple')
        plt.xlabel(column, fontsize=16)
        plt.ylabel('Count', fontsize=16)
        plt.show()


NameError: name 'discrete_df' is not defined

Top 10 correlated with TARGET_B

In [60]:
for column in discrete_df:
    if column in DB:
        plt.hist(discrete_df[column], bins=10, edgecolor="blue", color='purple')
        plt.xlabel(column, fontsize=16)
        plt.ylabel('Count', fontsize=16)
        plt.show()

NameError: name 'discrete_df' is not defined

Crosstabs

In [61]:
for column in discrete_df:
    if column in DB:
        table=pd.crosstab(discrete_df[column], targets['TARGET_B'])
        table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
        plt.xlabel(column)
        plt.ylabel('Proportion of Donation')


NameError: name 'discrete_df' is not defined

Heatmap of the correlation matric for the most correlated variables with Target B - discrete data

In [ ]:
m3 = heatmap(discrete_df, DB, targets)


Heatmap of the correlation matric for the most correlated variables with Target B - discrete data

High correlation within the RAMNT and HPV variables. We will have to fix the colinearity by keeping the varibale the most correlated to the target and discarding the others for our model.

Plotting the categorical data
Separating the categoricals with a lot of different unique values and thos with less unique values as they will be plotted differently

In [ ]:
low_count = []
high_count = []
for col in categoricals:
        if len(categoricals[col].value_counts()) >= 15:
            high_count.append(col)
        else:
            low_count.append(col)
print(len(low_count), ' categorical columns with less than 20 unique values')
print(len(high_count), 'categorical columns with mote than 20 unique values')
low = categoricals[low_count]
high = categoricals[high_count]

# Countplots

In [62]:
for column in low:
    plt.figure(figsize=(10,5))
    sns.countplot(x=column, data=low, palette='Greens')
    plt.show()

NameError: name 'low' is not defined

# Boxen plots
With TARGET_B

In [63]:
for column in low:      
    plt.figure(figsize=(6,3))
    plt.ylabel('Amounts')
    sns.boxenplot(x=low[column], y=targets['TARGET_D'], hue=targets['TARGET_B'], palette='Greens')
    plt.show()

NameError: name 'low' is not defined